In [ ]:
import os
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import random

import nltk
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
drive.mount("drive")
df = pd.read_csv('drive/MyDrive/NewsScraperProject/Datasets/ScalePartisan.csv', header=None)
booleandf = pd.read_csv('drive/MyDrive/NewsScraperProject/Datasets/BooleanPartisan.csv', header=None)
dftest = pd.read_csv('drive/MyDrive/NewsScraperProject/Datasets/articles1.csv')[:50]

display(df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at drive


In [ ]:
#clean and augment dataset
import re,string
from nltk.sentiment import SentimentIntensityAnalyzer
import ast
nltk.download('vader_lexicon')

def clean(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())

df[3] = df[3].apply(clean)
booleandf[3] = booleandf[3].apply(clean)

def nltkSentiment(sentence):
  #NLTK anaylzer provides 4 datapoints in a dictionary --> compound datapoint is used; range is -1 to 1 where -1 being most negative and 1 being most positive
  anaylzer = SentimentIntensityAnalyzer()
  return anaylzer.polarity_scores(sentence)["neg"]

df[4] = df[3].apply(nltkSentiment)
booleandf[4] = booleandf[3].apply(nltkSentiment)

display(df)
display(booleandf)

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


,0,1,2,3,4
0,Right,Fox News,1.0,White House backtracks after Biden raises eyeb...,0.000
1,Right,Fox News,1.0,Newsom signs controversial bill requiring ethn...,0.130
2,Right,Fox News,1.0,McConnell slams Garland for school board memo ...,0.117
3,Right,Fox News,1.0,Biden indicates he pressured emergency room to...,0.256
4,Right,Fox News,1.0,GOP demands Biden stop bullying airlines with ...,0.323
...,...,...,...,...,...
115,Control,NoSource,0.0,Light travels extremely quickly and is used in...,0.000
116,Control,NoSource,0.0,Control sentences help us establish factual se...,0.179
117,Control,NoSource,0.0,The length of sentences can greatly impact the...,0.000
118,Control,NoSource,0.0,Attention span is something that many news sou...,0.000


,0,1,2,3,4
0,Right,Fox News,1,White House backtracks after Biden raises eyeb...,0.000
1,Right,Fox News,1,Newsom signs controversial bill requiring ethn...,0.130
2,Right,Fox News,1,McConnell slams Garland for school board memo ...,0.117
3,Right,Fox News,1,Biden indicates he pressured emergency room to...,0.256
4,Right,Fox News,1,GOP demands Biden stop bullying airlines with ...,0.323
...,...,...,...,...,...
115,Control,NoSource,0,Light travels extremely quickly and is used in...,0.000
116,Control,NoSource,0,Control sentences help us establish factual se...,0.179
117,Control,NoSource,0,The length of sentences can greatly impact the...,0.000
118,Control,NoSource,0,Attention span is something that many news sou...,0.000


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
X_train,X_test,y_train,y_test = train_test_split(booleandf[3], booleandf[2], test_size=0.8)
# lab_enc = preprocessing.LabelEncoder()
# encoded = lab_enc.fit_transform(y_train)
# encodedtest = lab_enc.fit_transform(y_test)
from sklearn.linear_model import LogisticRegression
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])
# Fitting the model
regmodel = pipe.fit(X_train, y_train)
# Accuracy
prediction = regmodel.predict(X_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 66.67%


In [ ]:
regmodel.predict(["Biden refuses to assert privilege over Trump documents sought by January 6 committee"])

array([1])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
X_train,X_test,y_train,y_test = train_test_split(booleandf[3], booleandf[2], test_size=0.7)
pipe = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('model', RandomForestClassifier())])
model = pipe.fit(X_train, y_train)
prediction = model.predict(X_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 70.24%


In [ ]:
#Testing Model
import spacy
nlp = spacy.load("en_core_web_sm")

def modelTest(sentences):
    print("**************************** Start of Article Testing ****************************")
    predictions = model.predict(sentences)
    print(predictions)
    for i in range(len(sentences)):
      print(predictions[i])
      print(sentences[i])

    
def clean(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())  

dftest["content"] = dftest["content"].apply(clean)

for i in range(1):
  doc = nlp(dftest["content"][i])
  temp = [sent.text.strip() for sent in doc.sents]
  print(temp)
  modelTest(temp)
  

['WASHINGTON Congressional Republicans have a new fear when it comes to their health care lawsuit against the Obama administration', 'They might win', 'The incoming Trump administration could choose to no longer defend the executive branch against the suit which challenges the administration s authority to spend billions of dollars on health insurance subsidies for and Americans handing House Republicans a big victory on issues But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode leaving millions of people without access to health insurance before Republicans have prepared a replacement That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government To stave off that outcome Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law angering conservative voters who have been demanding an end t

In [ ]:
!wget https://people.cs.umass.edu/~miyyer/data/sample_ibc.tar.gz

--2021-10-19 00:24:10--  https://people.cs.umass.edu/~miyyer/data/sample_ibc.tar.gz
Resolving people.cs.umass.edu (people.cs.umass.edu)... 128.119.240.99
Connecting to people.cs.umass.edu (people.cs.umass.edu)|128.119.240.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235520 (230K) [application/x-gzip]
Saving to: ‘sample_ibc.tar.gz’

sample_ibc.tar.gz   100%[===================>] 230.00K  1.19MB/s    in 0.2s    

2021-10-19 00:24:11 (1.19 MB/s) - ‘sample_ibc.tar.gz’ saved [235520/235520]



In [ ]:
!tar -xf sample_ibc.tar.gz

In [ ]:
import pickle
[lib, con, neutral] = pickle.load(open('sampleData.pkl', 'rb'))

df_dataset = pd.DataFrame(columns=['text', 'label'])
# how to access sentence text
# print ('Liberal examples (out of ', len(lib), ' sentences): ')
for tree in lib:
  df_dataset = df_dataset.append({'text': tree.get_words(), 'label':1}, ignore_index=True)
  # print (tree.get_words())

# print ('\nConservative examples (out of ', len(con), ' sentences): ')
for tree in con:
  df_dataset = df_dataset.append({'text': tree.get_words(), 'label':-1}, ignore_index=True)
  # print (tree.get_words())

# print ('\nNeutral examples (out of ', len(neutral), ' sentences): ')
for tree in neutral:
  df_dataset = df_dataset.append({'text': tree.get_words(), 'label':0}, ignore_index=True)
  # print (tree.get_words())

print(df_dataset.head(101))




                                                  text label
0    First , government plays an important role thr...     1
1    And `` once we think positively about the Stat...     1
2    The narrow-minded rationality of the capitalis...     1
3    The resulting global warming kills Amazonian t...     1
4    If evolution is improvement and improvement is...     1
..                                                 ...   ...
96   In an op-ed for CNN , Otis argued that excessi...    -1
97   `` Central banks are grasping at straws , hopi...    -1
98   The possibility of a general loss of confidenc...    -1
99   During the 1920s , Great Britain , terrified o...    -1
100  This problem is exacerbated because using your...     0

[101 rows x 2 columns]


# Political Model


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df_pol[0], df_pol[2], test_size=0.2, random_state=42)
# count_vectorizer = CountVectorizer(stop_words='english')
print()

# # Fit and transform the training data 
# count_train = count_vectorizer.fit_transform(X_train) 

# # Transform the test set 
# count_test = count_vectorizer.transform(X_test)

# # Create a random forest Classifier. By convention, clf means 'Classifier'
clf_pol = RandomForestClassifier(n_jobs=2, random_state=0)

# Initialize the `tfidf_vectorizer` 
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7) 

# Fit and transform the training data 
tfidf_train = tfidf_vectorizer.fit_transform(X_train) 

# Transform the test set 
tfidf_test = tfidf_vectorizer.transform(X_test)


# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf_pol.fit(tfidf_train,y_train)
preds = clf_pol.predict(tfidf_test)

In [ ]:
pd.crosstab(y_test, preds, rownames=['Actual value'], colnames=['Predicted value'])
print(clf_pol.score(tfidf_test, y_test) *100, '%')

for i,value in enumerate(X_test):
  print(value,"'", preds[i],"'")